In [28]:
import requests
from lxml import etree
import xml.etree.ElementTree as ET
import zipfile, urllib
import pandas as pd

In [62]:
def get_data_urls(year_range=None):
    """
        Args:
            year_range - List: Parameter will take either a 2 value list or `None`. If parameter is a list, results will be restricted to years less than the first value and greater than the second.
                Example: [2003, 2010] - Files returned : 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
                Example: [2003, 2003] - Files returned : 2003
        Returns:
            all_data_links - List: List of links filtered by the requested `year_range`
    """
    url_base = 'https://exporter.nih.gov'
    url_search = url_base + '/ExPORTER_Catalog.aspx'

    url_text = requests.get(url_search).text

    parser = etree.XMLParser(recover=True)
    root = ET.fromstring(url_text, parser=parser)
    
    #### I stole this code from the internet but it will replace the prefix that gets added
    #### There's probably a better way to do this
    for elem in root.getiterator():
        if not hasattr(elem.tag, 'find'): continue
        i = elem.tag.find('}')
        if i >= 0:
            elem.tag = elem.tag[i+1:]
    ####
    data_2019 = [i.get('href') for i in root.findall('.//tr[@class="row_bg"]/td//a')]
    data_2019 = [i for i in data_2019  if ('CSVs' in i) and ('2019' in i)]
    data_2019 = [url_base + '/' + i for i in data_2019]
    ### Range + 1900 goes from 1985 to 2018    
    years = [1900 + i for i in range(85,119)]
    years = sorted(years, reverse = True)
    fy_links = [f'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY{year}.zip' for year in years]
    fy_links

    all_data_links = data_2019 + fy_links
    ### Make a copy because we modifying in place ya'll
    all_data_loop = all_data_links.copy()
    if year_range is not None:
        for link in all_data_loop:
            link_year = re.search('FY.{4}.*\.zip', link).group()
            ### Remove '.zip' then remove any '_\d' stuff that 2019 files have
            link_year = link_year.split('.')[0].split('_')[0]
            ### Remove 'FY'
            link_year = link_year[2:]
            link_year = int(link_year)
            if (link_year < year_range[0]) or (link_year > year_range[1]):
                ### modifying in place ya'll
                all_data_links.remove(link)
                
    return all_data_links

urls = get_data_urls([2019, 2019])
urls

['https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_053.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_052.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_051.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_050.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_049.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_048.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_047.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_046.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_045.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_044.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_043.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_042.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_041.zip',
 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_040.zip',
 'https://exporter.n

In [58]:
def extract_csv_from_zip(zip_link):
    zip_link = 'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY1993.zip'

    with zipfile.ZipFile(urllib.request.urlretrieve(zip_link, filename=None)[0], 'r') as archive:
        csv_data = archive.read('RePORTER_PRJ_C_FY1993.csv')
    rows = []
    for index, row in enumerate(csv_data.split(b'\n')):
        row = row.decode()
        if index == 0:
            header = row.split(',')
        else:
            rows.append(row.split(',')[:42])

    rows_df = pd.DataFrame(rows, columns=header)

    return rows_df

In [59]:
extract_csv_from_zip(urls[2])

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,PROJECT_TERMS,PROJECT_TITLE,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,3000201,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001049,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1049,NSS,,,,1
1,3000202,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001050,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1050,NSS,,,,1
2,3000203,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001051,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1051,NSS,,,,1
3,3000204,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001052,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1052,NSS,,,,1
4,3000205,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001053,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1053,NSS,,,,1
5,3000206,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001054,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1054,NSS,,,,1
6,3000207,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001055,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1055,NSS,,,,1
7,3000208,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001056,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1056,NSS,,,,1
8,3000209,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001057,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1057,NSS,,,,1
9,3000210,A03,AH,1,,1993-08-10T00:00:00,08/15/1993,06/30/1994,,A03AH001058,...,08/15/1993,06/30/1994,,PUBLIC HEALTH TRAINEESHIPS,1058,NSS,,,,1
